# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
data_file = ('../Output_data/Output_weather_data.csv')
vacation_file = pd.read_csv(data_file)
vacation_file.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness (%),Wind Speed (m/s),Country,Date
0,0,Clyde River,70.47,-68.59,44.60,87,40,8.05,CA,1596037471
1,1,Ürümqi,43.80,87.60,80.60,34,75,15.66,CN,1596037208
2,2,Bredasdorp,-34.53,20.04,66.20,55,0,13.87,ZA,1596037475
3,3,Puerto Madryn,-42.77,-65.04,47.77,52,0,11.83,AR,1596037477
4,4,Tasiilaq,65.61,-37.64,44.60,75,75,13.87,GL,1596037479


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
gmaps.configure(api_key=g_key)

In [13]:
#Storing lat/lng into locations
locations = vacation_file[["Lat", "Lng"]].astype(float)

#Convert humidity to float and store
humid_locale = vacation_file["Humidity"].astype(float)

In [14]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humid_locale, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
